In [ ]:
pip install datasets transformers peft

In [ ]:
import os
import torch
import numpy as np
import pandas as pd
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
from peft import LoraConfig, get_peft_model, PeftModel, TaskType
import matplotlib.pyplot as plt

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/machine learning/Copie de train.csv')
df.head()

,answer,input,output,instruction
0,163539,98296 + 65243,98296 + 65243 = 163539,Sum up 98296 + 65243
1,493238042432,895424 * 550843,895424 x 550843 = 493238042432,Could you please multiply 895424 * 550843
2,8.2,552^(1/3),cube root of 552 = 8.2,Determine the cube root of 552
3,48600,25270 + 23330,25270 + 23330 = 48600,Please compute the total of 25270 + 23330
4,18863800,2570 * 7340,2570 x 7340 = 18863800,Calculate the product of these two values: 257...


In [ ]:
df1 = pd.read_csv("/content/drive/MyDrive/machine learning/Copie de valid.csv")
df1.head()

,answer,input,output,instruction
0,30.52,28436^(1/3),cube root of 28436 = 30.52,Can you determine the cube root of 28436
1,13103,36188 - 23085,36188 - 23085 = 13103,Can you solve this equation: 36188 - 23085
2,441690054710841,11370419 * 38845539,11370419 x 38845539 = 441690054710841,Can you figure out the result of multiplying 1...
3,301.70515408259104,91026^(1/2),91026^(1/2) = 301.70515408259104,Can you figure out the square root of 91026
4,1.57,79960 / 51023,79960 / 51023 = 1.57,Please compute the quotient of 79960 / 51023


In [ ]:
df2 = pd.read_csv('/content/drive/MyDrive/machine learning/Copie de test.csv')
df2.head()

,answer,input,output,instruction
0,79779391/53755382,2198/9907 + 6849/5426,2198/9907 + 6849/5426 = 79779391/53755382,What is the addition of 2198/9907 + 6849/5426
1,40511/35670,125/246 + 91/145,125/246 + 91/145 = 40511/35670,Add these numbers: 125/246 + 91/145
2,768533,763572 + 4961,763572 + 4961 = 768533,What is the result when you add 763572 + 4961
3,67944/76825,149/350 / 439/912,149/350 / 439/912 = 67944/76825,What do you get when you divide 149/350 / 439/912
4,1.87,45247 / 24243,45247 / 24243 = 1.87,I need the quotient of 45247 / 24243


In [ ]:
895424 * 550843

493238042432

In [ ]:
import os
import torch
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
from peft import LoraConfig, get_peft_model, PeftModel, TaskType
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Load the datasets
def load_dataset(filepath):
    data = pd.read_csv(filepath)
    return Dataset.from_pandas(data)

# Preprocess the data
def preprocess_function(examples, tokenizer, max_length=512):
    inputs = examples["instruction"]
    targets = examples["answer"]
    inputs = [f"Question: {q}\nAnswer:" for q in inputs]
    model_inputs = tokenizer(inputs, max_length=max_length, truncation=True, padding="max_length")

    labels = tokenizer(targets, max_length=max_length, truncation=True, padding="max_length")["input_ids"]
    model_inputs["labels"] = labels
    return model_inputs

# Initialize LoRA configuration
def get_lora_model(base_model, lora_r=16, lora_alpha=32, lora_dropout=0.1):
    lora_config = LoraConfig(
        task_type=TaskType.CAUSAL_LM,
        inference_mode=False,
        r=lora_r,
        lora_alpha=lora_alpha,
        lora_dropout=lora_dropout,
    )
    return get_peft_model(base_model, lora_config)

# Plot and save results
def plot_results(train_loss, eval_loss, output_dir):
    plt.figure(figsize=(10, 6))
    plt.plot(train_loss, label='Train Loss', color='blue', marker='o')
    plt.plot(eval_loss, label='Validation Loss', color='orange', marker='o')
    plt.title('Training and Validation Loss')
    plt.xlabel('Steps')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid()
    plt.savefig(os.path.join(output_dir, 'training_results.png'))
    plt.show()

# Compute classification metrics
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = torch.argmax(torch.tensor(predictions), dim=-1).numpy()
    labels = labels.numpy()

    accuracy = accuracy_score(labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average="weighted")

    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }

# Main script
if __name__ == "__main__":
    train_path = "/content/drive/MyDrive/machine learning/Copie de train.csv"  # Replace with the path to your train dataset
    valid_path = "/content/drive/MyDrive/machine learning/Copie de valid.csv"  # Replace with the path to your validation dataset
    test_path = "/content/drive/MyDrive/machine learning/Copie de test.csv"    # Replace with the path to your test dataset
    model_name = "microsoft/phi-1"
    output_dir = "./lora_phi1b_finetuned"

    # Load the tokenizer and model
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")

    # Load and preprocess the datasets
    train_dataset = load_dataset(train_path)
    valid_dataset = load_dataset(valid_path)
    test_dataset = load_dataset(test_path)

    train_dataset = train_dataset.map(lambda x: preprocess_function(x, tokenizer), batched=True)
    valid_dataset = valid_dataset.map(lambda x: preprocess_function(x, tokenizer), batched=True)
    test_dataset = test_dataset.map(lambda x: preprocess_function(x, tokenizer), batched=True)

    # Apply LoRA to the model
    model = get_lora_model(model)
    model.print_trainable_parameters()

    # Define training arguments
    training_args = TrainingArguments(
        output_dir=output_dir,
        evaluation_strategy="steps",
        eval_steps=500,
        save_steps=500,
        save_total_limit=2,
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
        num_train_epochs=3,
        learning_rate=1e-4,
        weight_decay=0.01,
        logging_dir="./logs",
        logging_steps=100,
        report_to="tensorboard",
        fp16=True,
        push_to_hub=False
    )

    # Initialize the Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=valid_dataset,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics
    )

    # Train the model
    train_logs = trainer.train()

    # Extract training and validation losses for plotting
    train_loss = [log["loss"] for log in train_logs.log_history if "loss" in log]
    eval_loss = [log["eval_loss"] for log in train_logs.log_history if "eval_loss" in log]

    # Save the fine-tuned model
    trainer.save_model(output_dir)

    # Evaluate the model
    eval_results = trainer.evaluate(eval_dataset=test_dataset)
    print("Test Evaluation Results:", eval_results)

    # Plot and save resultsHF_TOKEN
    plot_results(train_loss, eval_loss, output_dir)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
